pip install googletrans==4.0.0-rc1

In [21]:
import pandas as pd
from langdetect import detect
from googletrans import Translator
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

In [9]:
job_data = pd.read_csv('glassdoor.csv')
job_data.head()

,benefits.benefitRatingDecimal,benefits.comments,benefits.highlights,benefits.numRatings,benefits.employerSummary,breadCrumbs,gaTrackerData.category,gaTrackerData.empId,gaTrackerData.empName,gaTrackerData.empSize,...,salary.currency.displayName,salary.currency.id,salary.currency.name,salary.currency.negativeTemplate,salary.currency.new,salary.currency.positiveTemplate,salary.currency.symbol,salary.lastSalaryDate,salary.salaries,wwfu
0,0.0,2801.0,2801.0,0,NaN,2801,-1,2183810,Modus Group,501-1000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2483.0,NaN
1,4.0,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-10-14T06:05:58,2484.0,NaN
2,0.0,2803.0,2803.0,0,NaN,2803,-1,1043373,Immobiliare.it,201-500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2485.0,NaN
3,0.0,2804.0,2804.0,0,NaN,2804,-1,2226886,Codup,1-50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2486.0,NaN
4,0.0,2805.0,2805.0,0,NaN,2805,-1,851555,Markelytics Solutions,201-500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2487.0,NaN


In [11]:
def clean_html_column(column):
    cleaned_values = []
    for value in column:
        if isinstance(value, str):  # Check if the value is a non-null string
            soup = BeautifulSoup(value, 'html.parser')
            cleaned_text = soup.get_text()
            cleaned_values.append(cleaned_text)
        else:
            cleaned_values.append(None)  # Handle NaN values by replacing them with None
    return cleaned_values

In [3]:
# Function to detect language
def detect_language(text):
    try:
        return detect(text)
    except:
        return "Unknown"  # Handle cases where language detection fails


In [27]:
# Function to translate text to English using Google Translate
def translate_to_english(row):
    #detected_language = row['Desc_Language']
    #text = row['Cleaned_Desc']
    detected_language = row[1]
    text = row[0]

    if detected_language == 'en':
        return text  # Text is already in English
    else:
        try:
            translator = Translator()
            translated = translator.translate(text, src=detected_language, dest='en')
            return translated.text
        except Exception as e:
            return f"Translation Error: {str(e)}"  # Handle translation errors

In [18]:
#job_data = job_data.sample(n=1000, random_state=42)
job_data['Cleaned_Desc'] = clean_html_column(job_data['job.description'])
job_data['Cleaned_Desc'] = job_data['Cleaned_Desc'].str.replace('\n', '')
job_data['Cleaned_Desc'].head()

/var/folders/bz/qyvd1hfs07v7gkh9fbvpy51m0000gn/T/ipykernel_65531/4184509349.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(value, 'html.parser')


0    We are the drivers of the future who offer a g...
1    Description:Join Boehringer Ingelheim and be p...
2    Dottori.it è il motore di ricerca che ti aiuta...
3    Codup is looking for a Senior SQA Engineer for...
4    Department: Research & OperationsVacancies: 3E...
Name: Cleaned_Desc, dtype: object

In [19]:
# Apply language detection to the 'Text' column
job_data['Desc_Language'] = job_data['Cleaned_Desc'].apply(detect_language)
job_data['Desc_Language'].value_counts().head(10)

Desc_Language
en    139134
de      7996
nl      5887
fr      5419
it      2226
es      1462
he       768
pl       576
pt       375
sv       372
Name: count, dtype: int64

In [29]:
max_workers = 4  # Adjust the number of workers as needed

# Use ThreadPoolExecutor to process rows concurrently
with ThreadPoolExecutor(max_workers) as executor:
    # Apply the translation function concurrently to each row
    results = list(executor.map(translate_to_english, job_data[["Cleaned_Desc","Desc_Language"]].itertuples(index=False)))

# Add the results to a new column in the DataFrame
job_data['Translated_Desc'] = results

In [31]:
# Display the DataFrame with detected languages and translated text
print(job_data[['Cleaned_Desc','Translated_Desc']].head(15))

                                         Cleaned_Desc  \
0   We are the drivers of the future who offer a g...   
1   Description:Join Boehringer Ingelheim and be p...   
2   Dottori.it è il motore di ricerca che ti aiuta...   
3   Codup is looking for a Senior SQA Engineer for...   
4   Department: Research & OperationsVacancies: 3E...   
5   About NCRNCR Corporation (NYSE: NCR) is a lead...   
6   תיאור המשרה:לחברת הייטק בינלאומית שפיתחה את הפ...   
7   Le descriptif de l’offre ci-dessous est en Ang...   
8   Technical Consultant will handle solution deli...   
9   ADLAB Solutions is looking for skillful candid...   
10  Some careers grow faster than others.If you’re...   
11  Overview:As the Installed Base Program Manager...   
12  Hiring Key Accounts Manager for an For a priva...   
13  Amaris est une société indépendante et interna...   
14  Why You'll Love CiscoWe change the World, you ...   

                                      Translated_Desc  
0   We are the drivers of the f

In [79]:
columns_to_keep = [
                  'gaTrackerData.industry','gaTrackerData.jobId.long',
                  'gaTrackerData.sector','header.employerName','header.jobTitle','header.location','header.locationType',
                  'map.country','map.lat','map.lng','map.location','salary.salaries',
                  'Cleaned_Desc','Desc_Language','Translated_Desc']

In [80]:
job_data[columns_to_keep].isna().sum()

gaTrackerData.industry      47405
gaTrackerData.jobId.long      246
gaTrackerData.jobTitle          0
gaTrackerData.sector        47373
header.employerName          3245
header.jobTitle                 0
header.location                 5
header.locationType            23
map.country                 50028
map.lat                         0
map.lng                         0
map.location                    5
salary.salaries             18141
Cleaned_Desc                    4
Desc_Language                   0
Translated_Desc                 0
dtype: int64

In [40]:
#get the missing coords
(job_data['map.lat']== 0).sum()

18701

In [41]:
(job_data['map.lng']== 0).sum()

18701

In [42]:
job_data[job_data['map.lat']== 0][['header.locationType']].value_counts().head(10)

header.locationType
N                      13138
S                       5171
C                        369
Name: count, dtype: int64

!pip install geopandas

In [44]:
import geopandas as gpd

# Load the shapefile
gdf = gpd.read_file('ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp')  # Replace with the actual path to your shapefile




column_names = gdf.columns
for col in column_names:
    print(col)

In [54]:
# Extract country names, latitude, and longitude
country_info = gdf[['NAME', 'LABEL_X', 'LABEL_Y']]  # Replace 'LATITUDE' and 'LONGITUDE' with the actual column names in your shapefile

# Display the data
print(country_info)

                NAME     LABEL_X    LABEL_Y
0           Zimbabwe   29.925444 -18.911640
1             Zambia   26.395298 -14.660804
2              Yemen   45.874383  15.328226
3            Vietnam  105.387292  21.715416
4          Venezuela  -64.599381   7.182476
..               ...         ...        ...
237      Afghanistan   66.496586  34.164262
238  Siachen Glacier   77.129553  35.340606
239       Antarctica   35.885455 -79.843222
240     Sint Maarten  -63.070133  18.040880
241           Tuvalu  179.209587  -8.513717

[242 rows x 3 columns]


In [66]:
missing_coord = job_data[(job_data['map.lat']== 0) & (job_data['header.locationType']=='S')][['header.location']]
missing_coord.head()

,header.location
6,Central
41,Eastern Province
54,Kiev
55,Tamil Nadu
66,East Flanders


In [85]:
job_data1 = job_data[columns_to_keep]
job_data1.shape

(165290, 16)

In [86]:
job_data1 = job_data1.merge(country_info, how='left',left_on='header.location', right_on='NAME')

# Apply conditional fill for coords when it's equal to 0
job_data1['map.lat'] = job_data1.apply(lambda row: row['LABEL_X'] if row['map.lat'] == 0 else row['map.lat'], axis=1)
job_data1['map.lng'] = job_data1.apply(lambda row: row['LABEL_Y'] if row['map.lng'] == 0 else row['map.lng'], axis=1)


job_data1 = job_data1.drop(columns=['LABEL_X','LABEL_Y','NAME'])


In [88]:
job_data1 = job_data1[~job_data1['Translated_Desc'].str.startswith("Translation Error:")]
#job_data1['map.lat'].isna().sum()
job_data1.shape

(140950, 16)

In [90]:
job_data1['map.lat'].isna().sum()

5021

In [93]:
job_data1 = job_data1[~job_data1['map.lat'].isna()]
job_data1.shape

(135929, 16)

In [89]:
print((job_data1['gaTrackerData.jobTitle'] == job_data1['header.jobTitle']).all())

True


with ThreadPoolExecutor(4) as executor:
    # Apply the translation function concurrently to each row
    results = list(executor.map(translate_to_english, job_data1[["header.jobTitle","title_lang"]].itertuples(index=False)))

# Add the results to a new column in the DataFrame
job_data1['job_title_in_en'] = results

In [91]:
job_data1.head()

,gaTrackerData.industry,gaTrackerData.jobId.long,gaTrackerData.jobTitle,gaTrackerData.sector,header.employerName,header.jobTitle,header.location,header.locationType,map.country,map.lat,map.lng,map.location,salary.salaries,Cleaned_Desc,Desc_Language,Translated_Desc
0,Investment Banking & Asset Management,3.227100e+09,Biogas Project Development Manager,Finance,Modus Group,Biogas Project Development Manager,Warsaw,C,Poland,52.2500,21.0000,Warsaw,2483.0,We are the drivers of the future who offer a g...,en,We are the drivers of the future who offer a g...
1,Biotech & Pharmaceuticals,3.406582e+09,Quality Manager - Boehringer Ingelheim Healthc...,Biotech & Pharmaceuticals,Boehringer Ingelheim,Quality Manager - Boehringer Ingelheim Healthc...,Porto,C,NaN,41.1500,-8.6167,Porto,2484.0,Description:Join Boehringer Ingelheim and be p...,en,Description:Join Boehringer Ingelheim and be p...
2,NaN,3.230738e+09,"Senior Software Engineer (PHP, Elixir, Python)",NaN,Immobiliare.it,"Senior Software Engineer (PHP, Elixir, Python)",Milan,C,Italy,45.4667,9.2000,Milan,2485.0,Dottori.it è il motore di ricerca che ti aiuta...,it,Dottori.it is the search engine that helps you...
3,NaN,3.406677e+09,Senior SQA Engineer,NaN,Codup,Senior SQA Engineer,Karāchi,C,PK,24.8667,67.0500,Karāchi,2486.0,Codup is looking for a Senior SQA Engineer for...,en,Codup is looking for a Senior SQA Engineer for...
4,Consulting,3.201516e+09,Research Manager,Business Services,MARKELYTICS SOLUTIONS INDIA,Research Manager,Bengaluru,C,IN,12.9670,77.5873,Bengaluru,2487.0,Department: Research & OperationsVacancies: 3E...,en,Department: Research & OperationsVacancies: 3E...


In [97]:
!pip install unidecode

In [ ]:
from unidecode import unidecode

# Apply unidecode to remove or replace non-UTF-8 characters
job_data1['Cleaned_Desc'] = job_data1['Cleaned_Desc'].apply(unidecode)
job_data1['Translated_Desc'] = job_data1['Translated_Desc'].apply(unidecode)

# Write the DataFrame to a CSV file
job_data1.to_csv('job_data_cleaned_desc.csv', index=False)

In [102]:
#cleaning job titles
unique_titles = list(set(job_data1['gaTrackerData.jobTitle']))
print(len(unique_titles)) #54K unique job titles

54228


In [116]:
from fuzzywuzzy import fuzz

/Users/rishi/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [156]:
#generated different terminology for roles using chat GPT for different job titles -
#the job profiles in the data were related to the below roles primarily
title_variations = {
    "data scientist": ["data scientist", "machine learning scientist", "data science"],
    "software developer": ["software engineer", "software developer", "programmer", "full stack developer", "full stack",
                          "back end developer", "python developer", "developer", "mobile developer",
                          "application developer", "java"],
    "data analyst": ["data analyst", "data specialist", "Analyst"],
    "research scientist": ["research scientist", "researcher", "scientist", "Quantitative Researcher",
                           "AI researcher","research development"],
    "business analyst": ["business analyst", "BA", "financial analyst", "BI", "finance analyst","bi developer",
                         "data warehouse", "power BI", "BI consultant", "BI analyst", "business intelligence"],
    "product manager": ["product manager", "product owner"],
    "project manager": ["project manager", "project coordinator", "team lead", "project lead", "program coordinator"],
    "data engineer": ["data engineer", "ETL developer", "data pipeline engineer"],
    "statistician": ["statistician", "data statistician", "stats analyst", "statistical programmer"],
    "database administrator": ["database administrator", "DBA", "database specialist", "database manager"],
    "database engineer": ["database engineer", "database developer", "DB engineer", "SQL Developer","storage engineer"],
    "machine learning engineer": ["machine learning engineer", "ML engineer", "AI engineer"],
    "Consultant": ["Consultant", "business consultant", "management consultant", "consulting", "technology consultant"],
    "business manager": ["business development manager", "BD manager", "sales manager", "business manager",
                        "sales executive", "business executive","solution manager","account manager"],
    "DevOps": ["DevOps Engineer", "SRE", "Automation Engineer", "Release Engineer", "DevOps"],
    "UI/UX Developer or Designer": ["UI/UX Developer", "UI Developer", "UX Designer", "front end",
                                   "php", "javascript", "web developer"],
    "Professor": ["Professor", "Faculty", "Associate professor", "assistant professor"]
}

In [165]:
import re
unqie_title_map = []
unique_titles_clean=[]
#remove special characters from title
for t in unique_titles:
    
    x = re.sub(r'[^a-zA-Z0-9\s]', ' ', t)
    #print(s)
    unique_titles_clean.append(x)
    unqie_title_map.append({
        "Original Title": t,
        "Title": x
    })

job_title_map1 = pd.DataFrame(unqie_title_map)
job_title_map1.head(10)    

,Original Title,Title
0,"Business Development Executive (Female, Living...",Business Development Executive Female Living...
1,BI Senior Developer,BI Senior Developer
2,"Research & Development, Industrial Design - P&...",Research Development Industrial Design P ...
3,Key Account Manager - Rare Disease & Oncology,Key Account Manager Rare Disease Oncology
4,Graduate Software Consultant,Graduate Software Consultant
5,Client Solutions Manager,Client Solutions Manager
6,Senior Consultant (f/m/x),Senior Consultant f m x
7,Senior Business Set Up Manager,Senior Business Set Up Manager
8,Support Engineer Power BI,Support Engineer Power BI
9,Project Manager (Station in Dongguan),Project Manager Station in Dongguan


In [170]:
import math
standardized_titles_list = []
i=0
for title in unique_titles_clean:
    clean_title = ''
    max_score = 0
    max_title = ''
    max_len = 0
    for k in title_variations:
        for p in title_variations[k]:
            similarity_score = fuzz.token_set_ratio(title.lower(), p.lower())
            if (similarity_score>=65):
                #giving weightage to matches that are longer
                if (max_score*math.sqrt(max_len)) < (similarity_score*math.sqrt(len(p))) :
                    
                    clean_title = k
                    max_title = p
                    max_len = len(p)
                    max_score = similarity_score
    
    standardized_titles_list.append({
        "Title": title,
        "Cleaned Title": clean_title,
        "matched variation": max_title,
        "similarity_score":max_score
    })
    

job_title_df = pd.DataFrame(standardized_titles_list)
job_title_df.head(10)    

0


,Title,Cleaned Title,matched variation,similarity_score
0,Business Development Executive Female Living...,business manager,business development manager,83
1,BI Senior Developer,business analyst,bi developer,100
2,Research Development Industrial Design P ...,research scientist,research development,100
3,Key Account Manager Rare Disease Oncology,business manager,account manager,100
4,Graduate Software Consultant,Consultant,Consultant,100
5,Client Solutions Manager,business manager,solution manager,80
6,Senior Consultant f m x,Consultant,Consultant,100
7,Senior Business Set Up Manager,business manager,business manager,100
8,Support Engineer Power BI,business analyst,power BI,100
9,Project Manager Station in Dongguan,project manager,project manager,100


In [171]:
job_title_df["similarity_score"].describe()

count    54228.000000
mean        90.028583
std         17.301130
min          0.000000
25%         82.000000
50%        100.000000
75%        100.000000
max        100.000000
Name: similarity_score, dtype: float64

In [174]:
(job_title_df["similarity_score"]==0).sum()

1182

In [175]:
job_title_df.to_excel("job_title_df.xlsx",index=False)

In [176]:
job_title_map2 = job_title_map1.merge(job_title_df,how='inner',on="Title")
job_title_map2.head()

,Original Title,Title,Cleaned Title,matched variation,similarity_score
0,"Business Development Executive (Female, Living...",Business Development Executive Female Living...,business manager,business development manager,83
1,BI Senior Developer,BI Senior Developer,business analyst,bi developer,100
2,"Research & Development, Industrial Design - P&...",Research Development Industrial Design P ...,research scientist,research development,100
3,Key Account Manager - Rare Disease & Oncology,Key Account Manager Rare Disease Oncology,business manager,account manager,100
4,Graduate Software Consultant,Graduate Software Consultant,Consultant,Consultant,100


In [178]:
job_data2 = job_data1.merge(job_title_map2,how='left',left_on='gaTrackerData.jobTitle',right_on='Original Title')
job_data2.drop(columns=['Original Title','Title'],inplace=True)
job_data2.head()

,gaTrackerData.industry,gaTrackerData.jobId.long,gaTrackerData.jobTitle,gaTrackerData.sector,header.employerName,header.jobTitle,header.location,header.locationType,map.country,map.lat,map.lng,map.location,salary.salaries,Cleaned_Desc,Desc_Language,Translated_Desc,Cleaned Title,matched variation,similarity_score
0,Investment Banking & Asset Management,3.227100e+09,Biogas Project Development Manager,Finance,Modus Group,Biogas Project Development Manager,Warsaw,C,Poland,52.2500,21.0000,Warsaw,2483.0,We are the drivers of the future who offer a g...,en,We are the drivers of the future who offer a g...,business manager,business development manager,81
1,Biotech & Pharmaceuticals,3.406582e+09,Quality Manager - Boehringer Ingelheim Healthc...,Biotech & Pharmaceuticals,Boehringer Ingelheim,Quality Manager - Boehringer Ingelheim Healthc...,Porto,C,NaN,41.1500,-8.6167,Porto,2484.0,Description:Join Boehringer Ingelheim and be p...,en,Description:Join Boehringer Ingelheim and be p...,business manager,BD manager,82
2,NaN,3.230738e+09,"Senior Software Engineer (PHP, Elixir, Python)",NaN,Immobiliare.it,"Senior Software Engineer (PHP, Elixir, Python)",Milan,C,Italy,45.4667,9.2000,Milan,2485.0,Dottori.it e il motore di ricerca che ti aiuta...,it,Dottori.it is the search engine that helps you...,software developer,software engineer,100
3,NaN,3.406677e+09,Senior SQA Engineer,NaN,Codup,Senior SQA Engineer,Karāchi,C,PK,24.8667,67.0500,Karāchi,2486.0,Codup is looking for a Senior SQA Engineer for...,en,Codup is looking for a Senior SQA Engineer for...,database engineer,DB engineer,84
4,Consulting,3.201516e+09,Research Manager,Business Services,MARKELYTICS SOLUTIONS INDIA,Research Manager,Bengaluru,C,IN,12.9670,77.5873,Bengaluru,2487.0,Department: Research & OperationsVacancies: 3E...,en,Department: Research & OperationsVacancies: 3E...,research scientist,Quantitative Researcher,67


In [180]:
job_data2['Cleaned Title'].value_counts()

Cleaned Title
software developer             22258
business manager               21060
project manager                15148
database engineer              13030
business analyst               12976
data analyst                   11339
product manager                 9365
data scientist                  5998
data engineer                   5410
DevOps                          5001
machine learning engineer       4834
Consultant                      4035
research scientist              3840
                                2110
database administrator          1413
Professor                        836
statistician                     705
UI/UX Developer or Designer      450
Name: count, dtype: int64

In [181]:
job_data2.to_csv('job_data_cleaned_tile.csv', index=False)